In [2]:
%run DataProcessing.ipynb

2024-08-15 13:04:39.681271
2004-08-15 13:04:39.681271


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


           Date  ^GSPC_change  ^GSPC_lag
0    2004-08-16     14.539917        NaN
1    2004-08-17      2.369995  14.539917
2    2004-08-18     13.460083   2.369995
3    2004-08-19     -3.940063  13.460083
4    2004-08-20      7.119995  -3.940063
...         ...           ...        ...
5029 2024-08-08     62.090332 -40.560059
5030 2024-08-09     37.219727  62.090332
5031 2024-08-12     25.100098  37.219727
5032 2024-08-13     65.379883  25.100098
5033 2024-08-14           NaN  65.379883

[5034 rows x 3 columns]
           Date  ^GSPC_change  ^GSPC_lag       ^IXIC        ^DJI      ^FCHI  \
0    2004-08-16     14.539917        NaN         NaN         NaN        NaN   
1    2004-08-17      2.369995  14.539917   32.680054  130.150391  42.409912   
2    2004-08-18     13.460083   2.369995   -4.599976    8.719727  15.120117   
3    2004-08-19     -3.940063  13.460083   38.250000  118.560547  29.449951   
4    2004-08-20      7.119995  -3.940063   -6.369995  -41.970703 -11.910156   
...      

C:\Users\chuac\AppData\Local\Temp\ipykernel_35212\1172888756.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In [3]:
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

### Create DataLoader instances

In [4]:
train_dataset = TensorDataset(X_train_seq, y_train_seq)
test_dataset = TensorDataset(X_test_seq, y_test_seq)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
for X_batch, y_batch in train_loader:
    print(X_batch.shape, y_batch.shape)

torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Size([32, 1])
torch.Size([32, 10, 9]) torch.Si

### Create LSTM class

In [6]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=9, hidden_size=50, num_layers=3, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Shape of each out:
        # (batch_size, sequence_length, hidden_size)
        outputs, _ = self.lstm(x)
        outputs = self.fc(outputs[:, -1, :])
        return outputs

# Instantiate the model
model = LSTMModel()

### Model Training

In [7]:
# Define loss function and optimizer
criterion = nn.MSELoss() # Mean Squared Error
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [8]:
# Define Mean Absolute Percentage Error (MAPE) function
def get_mape(y_true, y_pred):
    if y_true.shape == y_pred.shape: 
        epsilon = 1e-8  # Small value to avoid division by zero
        percentage_error = torch.abs((y_true - y_pred) / (y_true + epsilon))
        
        # Calculate the mean of absolute percentage error
        mape = torch.mean(percentage_error)
        
        return mape.item()

In [9]:
num_epochs = 70

model.train()

for epoch in range(num_epochs):
    model.train()
    # (Batch Size, Time Steps, Number of Features)
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad() 
        train_predictions = model(X_batch)
        mse = criterion(train_predictions, y_batch)
        mse.backward()
        optimizer.step()
    mape = get_mape(y_batch, train_predictions)
    print(f'Epoch [{epoch + 1}/{num_epochs}], MSE: {mse.item():.2f}, MAPE: {mape:.2f}')


Epoch [1/70], MSE: 1516.82, MAPE: 1.00
Epoch [2/70], MSE: 1509.59, MAPE: 1.01
Epoch [3/70], MSE: 1501.69, MAPE: 1.00
Epoch [4/70], MSE: 1478.15, MAPE: 0.99
Epoch [5/70], MSE: 1492.02, MAPE: 1.00
Epoch [6/70], MSE: 1487.64, MAPE: 0.99
Epoch [7/70], MSE: 1477.77, MAPE: 0.99
Epoch [8/70], MSE: 1462.38, MAPE: 0.98
Epoch [9/70], MSE: 1449.36, MAPE: 0.97
Epoch [10/70], MSE: 1419.55, MAPE: 0.95
Epoch [11/70], MSE: 1468.64, MAPE: 1.00
Epoch [12/70], MSE: 1368.63, MAPE: 0.94
Epoch [13/70], MSE: 1320.87, MAPE: 0.92
Epoch [14/70], MSE: 1287.41, MAPE: 0.90
Epoch [15/70], MSE: 1239.49, MAPE: 0.87
Epoch [16/70], MSE: 1186.22, MAPE: 0.84
Epoch [17/70], MSE: 1135.09, MAPE: 0.81
Epoch [18/70], MSE: 1048.99, MAPE: 0.76
Epoch [19/70], MSE: 1148.45, MAPE: 0.88
Epoch [20/70], MSE: 885.32, MAPE: 0.67
Epoch [21/70], MSE: 854.31, MAPE: 0.65
Epoch [22/70], MSE: 1317.33, MAPE: 0.90
Epoch [23/70], MSE: 1313.53, MAPE: 0.95
Epoch [24/70], MSE: 1220.87, MAPE: 0.93
Epoch [25/70], MSE: 942.26, MAPE: 0.69
Epoch [26/70

### Model Evaluation

In [10]:
model.eval()

with torch.no_grad():
    test_mse = []
    test_mape = []
    for X_batch, y_batch in test_loader:
        test_predictions = model(X_batch)
        mse = criterion(test_predictions, y_batch)
        mape = get_mape(y_batch, test_predictions)
        test_mse.append(mse.item())
        test_mape.append(mape)
    
    average_mse_loss = sum(test_mse) / len(test_mse)
    average_mape_loss = sum(test_mape) / len(test_mape)
    print(f'Average MSE: {average_mse_loss:.2f}, Average MAPE: {average_mape_loss:.2f}')

Average MSE: 2357.44, Average MAPE: 2.96


In [11]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'model_state_dict.pth')